<a href="https://colab.research.google.com/github/Kgs-Mathaba/TensorFlow_Development_ZTM/blob/main/08_Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Check GPU
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [4]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [5]:
## Get text dataset
## Kaggle's introduction to NLP dataset
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip data
unzip_data('nlp_getting_started.zip')

--2021-11-17 22:13:29--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.124.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.124.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-11-17 22:13:30 (120 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
import pandas as pd
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [7]:
train_df.shape, test_df.shape

((7613, 5), (3263, 4))

In [8]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [9]:
# shuffle training data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
# number of samples
len(train_df), len(test_df)

(7613, 3263)

In [13]:
# visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}', "(real diaster)" if target >0 else "(not real diaster)")
  print(f'Text:\n{text}\n')
  print('---\n')

Target: 0 (not real diaster)
Text:
I just nearly crashed my car typing 'Paul Rudd attacked by flying ants' into notes on my phone.

---

Target: 1 (real diaster)
Text:
daviesmutia: Breaking news! Unconfirmed! I just heard a loud bang nearby. in what appears to be a blast of wind from my neighbour's ass.

---

Target: 0 (not real diaster)
Text:
Are souls punished withåÊannihilation? http://t.co/c1QXJWeQQU http://t.co/Zhp0SOwXRy

---

Target: 0 (not real diaster)
Text:
I have an unexplainable desire to watch The Rescuers. #childhooddefined

---

Target: 1 (real diaster)
Text:
#hot  C-130 specially modified to land in a stadium and rescue hostages in Iran in 1980 http://t.co/FLqxd3q5pY #prebreak #best

---



In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)

In [16]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
# first ten sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [18]:
# Convert text into numbers
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [19]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens = False)

In [20]:
# Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length=max_length)

In [22]:
# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

In [23]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
# random sentece from training set
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence}\
      \n\n Vectorized version:')
text_vectorizer([random_sentence])

Original text:
 I'm in the shower and I went to go change the song and of course I get fucking electrocuted by the cord      

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  32,    4,    2, 1593,    7,    8,  492,    5,  112,  334,    2,
         528,    7,    6,  781]])>

In [25]:
# number of unique word in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words in vocab: {len(words_in_vocab)}')
print(f'5 Most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab: 10000
5 Most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [26]:
# Create embedding layer
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )


In [27]:
# Get a random sentence from the taining set
random_sentence = random.choice(train_sentences)
print(f'Original text: \n {random_sentence}\
      \n \n Embedded version:')

# Embed a random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 Whenever I have a meltdown and need someone @Becca_Caitlyn99 is always like 'leaving in 5' and I don't know how I got so lucky #blessed      
 
 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03047415, -0.01045531, -0.0221122 , ..., -0.00568329,
         -0.00634743,  0.00829549],
        [ 0.01341939,  0.031031  , -0.02295347, ..., -0.01926142,
         -0.03336096, -0.02535181],
        [-0.00890738, -0.04504935,  0.00169133, ...,  0.03818336,
          0.01735755, -0.02413387],
        ...,
        [-0.03002499,  0.0370759 ,  0.01127199, ...,  0.03327156,
          0.00079666, -0.04402564],
        [-0.04206401, -0.01122361,  0.04598551, ..., -0.00929584,
         -0.0147026 ,  0.02812784],
        [ 0.04432431, -0.01880922, -0.00098515, ..., -0.04114386,
         -0.00622085, -0.01470985]]], dtype=float32)>

In [29]:
# single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03047415, -0.01045531, -0.0221122 ,  0.01388529,  0.00960628,
        -0.04609536,  0.03830617,  0.00410444, -0.0323509 , -0.0268087 ,
        -0.02545495, -0.0493897 ,  0.03116698, -0.02699386, -0.02723563,
        -0.00034168, -0.04308871, -0.03628777, -0.01999387,  0.01297206,
         0.02976756,  0.00706214,  0.03493476,  0.00778385,  0.04582716,
        -0.01618642, -0.00243824, -0.02654077, -0.04184818, -0.04252115,
         0.02922598, -0.02257799, -0.04804319, -0.00481032, -0.02955805,
         0.01345923, -0.0485189 , -0.0149518 , -0.02004698, -0.04169607,
        -0.02820694,  0.03981808, -0.01660749, -0.02186451, -0.00423472,
         0.01082175, -0.02686421,  0.03925197, -0.04697285,  0.00367937,
         0.04829811, -0.04567821, -0.03488895, -0.03855337,  0.04759597,
        -0.04493768, -0.02366755, -0.01792104, -0.03290079, -0.02861979,
        -0.0450729 ,  0.02800279, -0.00849916, -0.01008553,  0.01952269,
  

In [34]:
# Model 0 (Baseline model)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB()),
])

# Fit pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [35]:
# Evaluate model
baseline_score = model_0.score(val_sentences, val_labels)
print(f'Baseline model has accuracy of: {baseline_score*100:2f}%')

Baseline model has accuracy of: 79.265092%


In [36]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)

In [37]:
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [41]:
# create an evaluation function for modelling experiments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):

  """
  Calculates model accuracy, precision, recall and f1-score for binary classification model.

  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100

  # Calculate model precision, recall and f1-score using wieghted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [42]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [43]:
# Model_1: A simple dense model
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Ctreate a directory to save a TensorBoard logs
SAVE_DIR = "model_logs"

In [51]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding 
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [52]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [53]:
# Compile model
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [54]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20211118-004227
Epoch 1/5
215/215 [==============================] - 4s 15ms/step - loss: 0.6094 - accuracy: 0.6935 - val_loss: 0.5328 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4423 - accuracy: 0.8209 - val_loss: 0.4705 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3478 - accuracy: 0.8605 - val_loss: 0.4548 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2853 - accuracy: 0.8918 - val_loss: 0.4612 - val_accuracy: 0.7953
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2389 - accuracy: 0.9113 - val_loss: 0.4776 - val_accuracy: 0.7887


In [55]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4776 - accuracy: 0.7887


[0.47763049602508545, 0.7887139320373535]

In [56]:
model_1_pred_probs = model_1.predict(val_sentences)

In [58]:
model_1_pred_probs[0]

array([0.33080786], dtype=float32)

In [60]:
# Convert model predictions probabilities to labels
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [62]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred = model_1_preds)

model_1_results

{'accuracy': 78.87139107611549,
 'f1': 0.7853694687698636,
 'precision': 0.7948464726843409,
 'recall': 0.7887139107611548}

In [63]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [65]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])